In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ICU-Tokenizer
from icu_tokenizer import Tokenizer
tokenizer = Tokenizer(lang='bengli')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 KB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ICU-Tokenizer: filename=icu_tokenizer-0.0.1-py3-none-any.whl size=13280 sha256=d4c1688a8956d33f78469800435891b98e8875a7daf5e99b9f31393ef6905405
  Stored in directory: /root/.cache/pip/wheels/8e/a2/a9/b6414263fa62b478a0bac882d8ad7801018fd6c6cb4b32f6ed
  Created wheel for PyICU: filename=PyICU-2.10.2-cp38-cp38-linux_x86_64.whl size=1396697 sha256=472230825602ca44d825db3b5b68eb85d81b952245185ee5f0ee3f6f039a73df
  Stored in directory: /root/.cache/pip/wheels/d8/30/74/41fd0a7e2e17f8f92d5a4584c2aad30d8235c8cfa63a13742f
Successfully built ICU-Tokenizer PyICU


In [ ]:
import pandas as pd
import numpy as np
import json
import string
import re
from pprint import pprint
print(pd.__version__)

1.3.5


In [ ]:
test = pd.read_csv("/content/drive/My Drive/Data_set/preprocess data/filename.csv")


In [ ]:
corpus = []
for text in test['finalsummery']:
  words = [word for word in tokenizer.tokenize(text)]
  corpus.append(words)

# Fasttxt Model

In [ ]:
from gensim.models import FastText
model_ted = FastText(corpus, size=100, window=5, min_count=5, workers=4,sg=1)

In [ ]:
model_ted.save('/content/drive/MyDrive/Data_set/pre-trained model/Fasttext/saved_model_gensim2')

In [ ]:
from gensim.models.fasttext import FastText
modelpath = '/content/drive/MyDrive/Data_set/pre-trained model/Fasttext/saved_model_gensim'
model = FastText.load(modelpath)

FileNotFoundError: ignored

In [ ]:
from gensim.models.fasttext import FastText
modelpath = '/content/drive/MyDrive/Data_set/pre-trained model/Fasttext/saved_model_gensim2'
model2 = FastText.load(modelpath)

In [ ]:
model.wv.most_similar(",")

[('ও', 0.710696816444397),
 ('।', 0.662300705909729),
 ('‘', 0.6393415927886963),
 ('এবং', 0.6067748069763184),
 ('নাদেলের', 0.5612422227859497),
 ('দেলওয়ার', 0.5609130859375),
 ('‑', 0.5408311486244202),
 ('বলেন', 0.5379682779312134),
 ('নাদেল', 0.537921667098999),
 ('বায়জীদ', 0.5317046642303467)]

In [ ]:
model2.wv.most_similar("!")

[('”', 0.7188056707382202),
 ('ইয়ুথ', 0.6466854810714722),
 ('ইয়েট', 0.6423752307891846),
 ('সিনক্লেয়ার', 0.6392045021057129),
 ('স্টাডিতে', 0.6376911401748657),
 ('ডিবেট', 0.6371051073074341),
 ('এমফিথিয়েটারে', 0.6349629759788513),
 ('থিয়েটার', 0.6333267688751221),
 ('মেরিডিয়ান', 0.6329158544540405),
 ('এনভায়রনমেন্টাল', 0.6310501098632812)]

In [ ]:
model2

# Glove Model

In [ ]:
np.savetxt("corpus.txt", corpus, delimiter=", ", fmt="% s")


In [ ]:
model_ted.save('/content/drive/MyDrive/Data_set/pre-trained model/Fasttext/saved_model_gensim')

In [ ]:
!cp "/content/corpus.txt" "/content/glove"

# BUt if you still want to move them not copy , Here's solution for that :
# !mv "/content/corpus.txt" "/content/glove"

In [ ]:
!git clone http://github.com/stanfordnlp/glove

In [ ]:
%cd glove

In [ ]:
!make

corpus.txt should be exist in glove folder

before run demo.sh edit the file->write corpus.txt in the place of text8

In [ ]:
!/content/glove/demo.sh
# ! path/demo.sh

In [ ]:
test = pd.read_fwf('/content/drive/MyDrive/Data_set/pre-trained model/Glove/corpus.txt')
pprint(test)

In [ ]:
type(test)

pandas.core.frame.DataFrame

# TF word2vec

In [ ]:
import tensorflow as tf

# Preprocess the data
# vocab_size = 10000  # Size of the vocabulary
word_sequences = [["the", "cat", "sat", "on", "the", "mat"], ["the", "dog", "barked", "at", "the", "mailman"]]
labels = [["cat", "sat", "on", "the", "mat", "."], ["dog", "barked", "at", "the", "mailman", "."]]

# Create a mapping from words to integers
word_index = {}
for seq in word_sequences:
    for word in seq:
        if word not in word_index:
            word_index[word] = len(word_index)

# # Create a mapping from integers to words
# index_word = {index: word for word, index in word_index.items()}
word_index["<UNK>"] = len(word_index)
index_word = {index: word for word, index in word_index.items()}
vocab_size = len(word_index)

# # Create the input and label data
# X = [[word_index[word] for word in seq] for seq in word_sequences]
# Y = [[word_index[word] for word in seq] for seq in labels]

X = [[word_index.get(word, word_index["<UNK>"]) for word in seq] for seq in word_sequences]
Y = [[word_index.get(word, word_index["<UNK>"]) for word in seq] for seq in labels]

# Define the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128, input_length=6))
model.add(tf.keras.layers.LSTM(128))
model.add(tf.keras.layers.Dense(6, activation="softmax"))

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam")

# Fit the model
model.fit(X, Y, batch_size=32, epochs=10)
model.summary()
print(vocab_size)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 48.3474
Epoch 2/10
1/1 [==============================] - 0s 17ms/step - loss: 48.1884
Epoch 3/10
1/1 [==============================] - 0s 22ms/step - loss: 48.0289
Epoch 4/10
1/1 [==============================] - 0s 20ms/step - loss: 47.8623
Epoch 5/10
1/1 [==============================] - 0s 20ms/step - loss: 47.6821
Epoch 6/10
1/1 [==============================] - 0s 15ms/step - loss: 47.4819
Epoch 7/10
1/1 [==============================] - 0s 18ms/step - loss: 47.2549
Epoch 8/10
1/1 [==============================] - 0s 16ms/step - loss: 46.9943
Epoch 9/10
1/1 [==============================] - 0s 18ms/step - loss: 46.6935
Epoch 10/10
1/1 [==============================] - 0s 17ms/step - loss: 46.3467
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 6, 128)            1

In [ ]:
model.predict(X)

1/1 [==============================] - 0s 432ms/step


array([[0.13929088, 0.16535972, 0.16941279, 0.1312256 , 0.1702401 ,
        0.22447091],
       [0.13603178, 0.17063949, 0.17425424, 0.12578954, 0.17132857,
        0.22195631]], dtype=float32)

In [ ]:
import numpy as np

# Generate embeddings for the word "cat"
cat_embedding = model.predict(np.array([[word_index["cat"]]]))

# Generate embeddings for the word "dog"
dog_embedding = model.predict(np.array([[word_index["dog"]]]))

# Compute the cosine similarity between the two embeddings
similarity = np.dot(cat_embedding, dog_embedding) / (np.linalg.norm(cat_embedding) * np.linalg.norm(dog_embedding))
print(similarity)

1/1 [==============================] - 0s 20ms/step


ValueError: ignored